In [31]:
import string
import csv
import collections
import nltk
from nltk.corpus import stopwords
import operator
from collections import OrderedDict
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io
import os
import math
from textblob import TextBlob as tb

english_stops = []


def clean_tokens(tokens):
    """ Lowercases, takes out punct and stopwords and short strings """
    return [token.lower() for token in tokens if (token not in string.punctuation) and
               	(token.lower() not in english_stops) and len(token) > 2]

def get_stopwords():
    enc = 'utf-8'
    with open('stopword_file.csv', 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    english_stops = [i[0] for i in keywords]
    #print ( english_stops)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
def get_cleanTokens(directory):
    cleanTokens = []
    wordcount = {} 
    for filename in os.listdir(directory):
        text = get_text("KeywordDocs/" + filename)
        tokens = nltk.word_tokenize(text)
        cleanTokens.extend(clean_tokens(tokens))
    return cleanTokens
     
def create_freqList(cleanTokens):
    for tok in cleanTokens:
        if tok not in wordcount:
            wordcount[tok] = 1
        else:
            wordcount[tok] += 1
        
    sorted_wordCount = OrderedDict(sorted(wordcount.items(), key=operator.itemgetter(1), reverse=True))
    write_csv(sorted_wordCount)

def write_csv(sorted_wordCount):
    enc = 'utf-8'
    if not os.path.isfile('frequency.csv'):
        with open('frequency.csv', 'w', encoding = enc) as f:
            for key in sorted_wordCount.keys():
                f.write("%s,\n"%(key))
    else:
        with open('frequency.csv', 'a+', encoding = enc) as f:
            for key in sorted_wordCount.keys():
                f.write("%s,\n"%(key))


def get_text(filename):
    fp = open(filename, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data



def tf(word, blob):
    return (float)(blob.words.count(word)) / (float)(len(blob.words))

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist): return math.log(len(bloblist) / (float)(1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)


def create_tfIdfList(document):
    totalLength, splitLength = len(document), int(len(document)/400)
    bloblist = [ document[i:i+splitLength] for i in range(0, totalLength, splitLength) ]
    for i, blob in enumerate(bloblist):
        blob = tb(blob)
        print("Top words in document {}".format(i + 1))
        scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        for word, score in sorted_words: 
            print("Word: {}, TF-IDF: {}".format(word, round(score, 5)))


if __name__ == '__main__':
    get_stopwords()
    clean_tokens = get_cleanTokens("KeywordDocs")
    #create_freqList(clean_tokens)
    create_tfIdfList(' '.join(clean_tokens))
    

Top words in document 1
Word: deloitte, TF-IDF: 0.05337
Word: dttl, TF-IDF: 0.04652
Word: outreach, TF-IDF: 0.03236
Word: dialogue, TF-IDF: 0.03101
Word: services, TF-IDF: 0.02171
Word: this, TF-IDF: 0.02119
Word: involved, TF-IDF: 0.02048
Word: member, TF-IDF: 0.02048
Word: pragmatic, TF-IDF: 0.01956
Word: concerning, TF-IDF: 0.01956
Word: assisted, TF-IDF: 0.01956
Word: thoughts, TF-IDF: 0.01956
Word: mentorship, TF-IDF: 0.01956
Word: inspirational, TF-IDF: 0.01956
Word: finally, TF-IDF: 0.01956
Word: communities, TF-IDF: 0.01806
Word: forum, TF-IDF: 0.01707
Word: foreword, TF-IDF: 0.017
Word: contributions, TF-IDF: 0.017
Word: prepared, TF-IDF: 0.01618
Word: august, TF-IDF: 0.01618
Word: sessions, TF-IDF: 0.01618
Word: enriched, TF-IDF: 0.01618
Word: played, TF-IDF: 0.01618
Word: invaluable, TF-IDF: 0.01618
Word: fourth, TF-IDF: 0.01618
Word: industrial, TF-IDF: 0.01618
Word: revolution, TF-IDF: 0.01618
Word: grateful, TF-IDF: 0.01618
Word: deloitte1, TF-IDF: 0.01618
Word: generous,

Word: fourth, TF-IDF: 0.0555
Word: context, TF-IDF: 0.0463
Word: seismic, TF-IDF: 0.03355
Word: approach…………………………………………………….………………………………………………………………………………………………..……………, TF-IDF: 0.03355
Word: payments…………………………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: insurance…………………………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: banking…………………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: lending………………………………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: management…………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: crowdfunding…………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: infrastructure………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: acknowledgements……………………………………………………………………………………………………………………………………………………………………………, TF-IDF: 0.03355
Word: characterized, TF-IDF:

Word: 2017, TF-IDF: 0.07505
Word: fintechs, TF-IDF: 0.04161
Word: succeeded, TF-IDF: 0.03786
Word: workshops, TF-IDF: 0.03674
Word: context, TF-IDF: 0.03484
Word: march, TF-IDF: 0.03291
Word: provisioned, TF-IDF: 0.03132
Word: consumed, TF-IDF: 0.03002
Word: leaders, TF-IDF: 0.02931
Word: april, TF-IDF: 0.02891
Word: experts, TF-IDF: 0.02814
Word: structured, TF-IDF: 0.02506
Word: innovators, TF-IDF: 0.02396
Word: approach, TF-IDF: 0.02297
Word: innovations, TF-IDF: 0.02231
Word: dominant, TF-IDF: 0.02178
Word: held, TF-IDF: 0.02142
Word: have, TF-IDF: 0.02125
Word: subject, TF-IDF: 0.02074
Word: matter, TF-IDF: 0.02042
Word: produce, TF-IDF: 0.01893
Word: davos‐klosters, TF-IDF: 0.01893
Word: francisco, TF-IDF: 0.01893
Word: shenzhen, TF-IDF: 0.01893
Word: toronto, TF-IDF: 0.01893
Word: came, TF-IDF: 0.01893
Word: overtaking, TF-IDF: 0.01893
Word: fallen, TF-IDF: 0.01893
Word: seized, TF-IDF: 0.01893
Word: usa, TF-IDF: 0.01763
Word: c‐suite, TF-IDF: 0.01748
Word: subsector, TF-IDF: 0.

Word: commoditization, TF-IDF: 0.03058
Word: differentiation, TF-IDF: 0.02884
Word: landscape, TF-IDF: 0.02563
Word: institutions, TF-IDF: 0.02139
Word: forces, TF-IDF: 0.02137
Word: neither, TF-IDF: 0.02071
Word: financial, TF-IDF: 0.02013
Word: predicated, TF-IDF: 0.01912
Word: redistributing, TF-IDF: 0.01912
Word: hyper‐scaled, TF-IDF: 0.01912
Word: hyper‐focused, TF-IDF: 0.01912
Word: systemically, TF-IDF: 0.01912
Word: important, TF-IDF: 0.01841
Word: entering, TF-IDF: 0.018
Word: redistribution, TF-IDF: 0.018
Word: bionic, TF-IDF: 0.018
Word: labour, TF-IDF: 0.018
Word: resemble, TF-IDF: 0.018
Word: serious, TF-IDF: 0.01713
Word: tempo, TF-IDF: 0.01713
Word: enriched, TF-IDF: 0.01713
Word: accelerating, TF-IDF: 0.01641
Word: grounds, TF-IDF: 0.01641
Word: pure, TF-IDF: 0.01641
Word: humans, TF-IDF: 0.01641
Word: regions, TF-IDF: 0.01641
Word: single, TF-IDF: 0.01611
Word: static, TF-IDF: 0.01581
Word: partnerships, TF-IDF: 0.01581
Word: increasingly, TF-IDF: 0.01581
Word: will, T

Word: organizations, TF-IDF: 0.04986
Word: chains, TF-IDF: 0.04447
Word: pools, TF-IDF: 0.04447
Word: customer‐facing, TF-IDF: 0.03522
Word: profit, TF-IDF: 0.03079
Word: thereby, TF-IDF: 0.0305
Word: example, TF-IDF: 0.02871
Word: start‐ups, TF-IDF: 0.02449
Word: companies, TF-IDF: 0.02416
Word: technological, TF-IDF: 0.02296
Word: chain, TF-IDF: 0.02138
Word: company‐wide, TF-IDF: 0.02129
Word: jigsaw, TF-IDF: 0.02129
Word: separately, TF-IDF: 0.02129
Word: business‐to‐business, TF-IDF: 0.02129
Word: catalysts, TF-IDF: 0.02129
Word: intra‐value, TF-IDF: 0.02129
Word: inter‐value, TF-IDF: 0.02129
Word: trōv, TF-IDF: 0.02129
Word: proposing, TF-IDF: 0.02129
Word: exchange‐traded, TF-IDF: 0.02129
Word: value, TF-IDF: 0.02071
Word: disruption, TF-IDF: 0.02056
Word: thinking, TF-IDF: 0.01966
Word: redistributing, TF-IDF: 0.01966
Word: permissions, TF-IDF: 0.0185
Word: treated, TF-IDF: 0.0185
Word: offices, TF-IDF: 0.0185
Word: redistribution, TF-IDF: 0.0185
Word: bypassing, TF-IDF: 0.0185

...

Word: tradingsolutions, TF-IDF: 0.02031
Word: combines, TF-IDF: 0.02031
Word: neural, TF-IDF: 0.02031
Word: genetic, TF-IDF: 0.02031
Word: sophisticated, TF-IDF: 0.01962
Word: generating, TF-IDF: 0.01876
Word: disappear, TF-IDF: 0.01876
Word: topological, TF-IDF: 0.01876
Word: 158, TF-IDF: 0.01876
Word: intervened, TF-IDF: 0.01876
Word: analysis, TF-IDF: 0.01847
Word: various, TF-IDF: 0.01806
Word: predicts, TF-IDF: 0.01765
Word: eliminated, TF-IDF: 0.01765
Word: execution, TF-IDF: 0.01711
Word: leverages, TF-IDF: 0.0168
Word: parameters, TF-IDF: 0.0168
Word: reverting, TF-IDF: 0.0168
Word: utilisation, TF-IDF: 0.0168
Word: readable, TF-IDF: 0.0168
Word: sources, TF-IDF: 0.01643
Word: incorporating, TF-IDF: 0.0161
Word: gradually, TF-IDF: 0.0161
Word: optimise, TF-IDF: 0.0161
Word: universe, TF-IDF: 0.01551
Word: feeds, TF-IDF: 0.01551
Word: real-life, TF-IDF: 0.01551
Word: analysed, TF-IDF: 0.01551
Word: simultaneously, TF-IDF: 0.01551
Word: among, TF-IDF: 0.01536
Word: essential, TF-

Word: discovered, TF-IDF: 0.01501
Word: proliferate, TF-IDF: 0.01459
Word: alternatively, TF-IDF: 0.01459
Word: workforce, TF-IDF: 0.01421
Word: appetite, TF-IDF: 0.01421
Word: opening, TF-IDF: 0.01421
Word: intermediating, TF-IDF: 0.01387
Word: positions, TF-IDF: 0.01387
Word: broaden, TF-IDF: 0.01387
Word: react, TF-IDF: 0.01355
Word: uncertainty, TF-IDF: 0.01299
Word: machines, TF-IDF: 0.01299
Word: supply, TF-IDF: 0.01299
Word: redefine, TF-IDF: 0.01249
Word: variety, TF-IDF: 0.01249
Word: smarter, TF-IDF: 0.01227
Word: crisis, TF-IDF: 0.01227
Word: buyer, TF-IDF: 0.01227
Word: extend, TF-IDF: 0.01205
Word: markets, TF-IDF: 0.0119
Word: connecting, TF-IDF: 0.01185
Word: generated, TF-IDF: 0.01166
Word: intermediation, TF-IDF: 0.01166
Word: playing, TF-IDF: 0.01166
Word: connections, TF-IDF: 0.01166
Word: emerged, TF-IDF: 0.01147
Word: limiting, TF-IDF: 0.0113
Word: discover, TF-IDF: 0.01113
Word: remaining, TF-IDF: 0.01081
Word: field, TF-IDF: 0.01081
Word: themselves, TF-IDF: 0.01

Word: provisioning, TF-IDF: 0.0109
Word: interaction, TF-IDF: 0.01046
Word: advantages, TF-IDF: 0.01033
Word: sophisticated, TF-IDF: 0.0102
Word: efficient, TF-IDF: 0.00995
Word: assess, TF-IDF: 0.00995
Word: characteristics, TF-IDF: 0.00983
Word: highly, TF-IDF: 0.0096
Word: available, TF-IDF: 0.0096
Word: automation, TF-IDF: 0.00949
Word: counterparty, TF-IDF: 0.00949
Word: cases, TF-IDF: 0.00949
Word: leading, TF-IDF: 0.00939
Word: trusted, TF-IDF: 0.00928
Word: networks, TF-IDF: 0.00918
Word: established, TF-IDF: 0.00909
Word: best, TF-IDF: 0.00899
Word: visibility, TF-IDF: 0.0089
Word: points, TF-IDF: 0.0088
Word: price, TF-IDF: 0.00871
Word: manual, TF-IDF: 0.00863
Word: connected, TF-IDF: 0.00854
Word: only, TF-IDF: 0.00806
Word: distributed, TF-IDF: 0.00791
Word: making, TF-IDF: 0.00784
Word: more, TF-IDF: 0.00779
Word: small, TF-IDF: 0.0077
Word: facilitate, TF-IDF: 0.00744
Word: limited, TF-IDF: 0.00744
Word: activities, TF-IDF: 0.00725
Word: existing, TF-IDF: 0.00713
Word: h

Word: counterparties, TF-IDF: 0.04286
Word: buyers, TF-IDF: 0.03918
Word: sellers, TF-IDF: 0.03918
Word: specialisation, TF-IDF: 0.03624
Word: scenario, TF-IDF: 0.03356
Word: large, TF-IDF: 0.03214
Word: small, TF-IDF: 0.03196
Word: adopting, TF-IDF: 0.03051
Word: diversification, TF-IDF: 0.029
Word: newer, TF-IDF: 0.02834
Word: best, TF-IDF: 0.02797
Word: reputation, TF-IDF: 0.02716
Word: intermediaries, TF-IDF: 0.02706
Word: transparency, TF-IDF: 0.02606
Word: risks, TF-IDF: 0.02604
Word: connecting, TF-IDF: 0.02478
Word: without, TF-IDF: 0.02418
Word: institutions, TF-IDF: 0.02263
Word: provisioning, TF-IDF: 0.02261
Word: top-level, TF-IDF: 0.0219
Word: line-level, TF-IDF: 0.0219
Word: pre-defined, TF-IDF: 0.0219
Word: constricted, TF-IDF: 0.0219
Word: broadest, TF-IDF: 0.0219
Word: specialization, TF-IDF: 0.0219
Word: smaller, TF-IDF: 0.02143
Word: buy-in, TF-IDF: 0.02023
Word: 169, TF-IDF: 0.02023
Word: counterparty, TF-IDF: 0.01969
Word: size, TF-IDF: 0.01905
Word: 168, TF-IDF: 0

Word: resulting, TF-IDF: 0.01209
Word: errors, TF-IDF: 0.01189
Word: connections, TF-IDF: 0.01189
Word: playing, TF-IDF: 0.01189
Word: another, TF-IDF: 0.01171
Word: professional, TF-IDF: 0.01119
Word: involved, TF-IDF: 0.01119
Word: pricing, TF-IDF: 0.01119
Word: provisioning, TF-IDF: 0.01103
Word: remaining, TF-IDF: 0.01103
Word: advice, TF-IDF: 0.01103
Word: field, TF-IDF: 0.01103
Word: scenarios, TF-IDF: 0.01073
Word: less, TF-IDF: 0.01073
Word: sophisticated, TF-IDF: 0.01032
Word: connection, TF-IDF: 0.01032
Word: safe, TF-IDF: 0.01019
Word: beyond, TF-IDF: 0.01007
Word: efficient, TF-IDF: 0.01007
Word: ability, TF-IDF: 0.00982
Word: become, TF-IDF: 0.0098
Word: institutional, TF-IDF: 0.00972
Word: clients, TF-IDF: 0.00972
Word: differentiate, TF-IDF: 0.00972
Word: likely, TF-IDF: 0.00961
Word: counterparty, TF-IDF: 0.00961
Word: offerings, TF-IDF: 0.0095
Word: important, TF-IDF: 0.0095
Word: does, TF-IDF: 0.00929
Word: platforms, TF-IDF: 0.00927
Word: worth, TF-IDF: 0.0092
Word: 

In [24]:
! pip install textblob